In [1]:
import onnxruntime
import numpy as np
from sklearn.preprocessing import normalize

from tokenization_enc_dec_encn import EncDecTokenizer

In [2]:
tokenizer = EncDecTokenizer('./encn/vocab.txt')

In [3]:
ses = onnxruntime.InferenceSession('./cpm_2_0_encoder_onnx_q/__MODEL_PROTO.onnx')

In [17]:
%%time
outputs = ses.run(None, {'input_1': np.array([[5503, 145, 44, 44]], dtype=np.int32)})[0]

CPU times: user 21 s, sys: 232 ms, total: 21.2 s
Wall time: 657 ms


In [18]:
outputs.shape

(1, 4, 4096)

In [21]:
outputs[0, 0, :10]

array([ 0.41527578,  0.06284113, -0.05914639,  0.26560524, -0.04305364,
       -0.0499084 , -0.00426697, -0.04841684, -0.18556522, -0.10881829],
      dtype=float32)

In [23]:
outputs[0, 1, :10]

array([-0.14076309,  0.27944347,  0.04254135,  0.17435642, -0.09576027,
       -0.10378038, -0.22219954, -0.13227384,  0.65702987, -0.19214422],
      dtype=float32)

In [24]:
outputs[0, 2, :10]

array([-0.07036787,  0.14362228, -0.0479506 ,  0.01865175, -0.03610381,
       -0.11935142,  0.02762288, -0.00787069, -0.05360343, -0.06178436],
      dtype=float32)

In [5]:
input_text = '''当地时间9月6日是美国劳工节，但就在这一天，上千万美国劳动者却陷入新的困境。因为美国政府为疫情期间失业者提供的主要救助同日到期，而且白宫表示没有进一步延长救助的计划。
在德尔塔变异株已把美国推入新一轮疫情的背景下，失业救济的突然“断供”意味着有上千万美国人将全部或部分失去他们的生活来源。'''
input_ids = np.array([tokenizer.encode(input_text)], dtype=np.int32)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.024 seconds.
Prefix dict has been built successfully.


In [6]:
%%time
outputs = ses.run(None, {'input_1': input_ids})[0]

CPU times: user 1min 22s, sys: 0 ns, total: 1min 22s
Wall time: 2.55 s


In [11]:
def encode(text):
    token = tokenizer.encode(text)
    input_ids = np.array([token], dtype=np.int32)
    # input_ids = np.array([[tokenizer.encode(x)[0] for x in text]], dtype=np.int32)
    outputs = ses.run(None, {'input_1': input_ids})[0]
    # print(token, w)
    outputs = np.sum(outputs, axis=1)
    # outputs = np.mean(outputs, axis=1)
    # outputs = np.max(outputs, axis=1)
    # outputs = outputs[:, 0, :]
    outputs = normalize(outputs)
    return outputs

In [12]:
np.sum(encode('添加提醒') * encode('添加提醒-'))

0.4328678

In [13]:
np.sum(encode('成都') * encode('重庆'))

0.64158475

In [ ]:
np.sum(encode('成都') * encode('重庆'))